##### Copyright 2020 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Recommenders: Quickstart

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/recommenders/quickstart"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/recommenders/blob/main/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/recommenders/docs/examples/quickstart.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In this tutorial, we build a simple matrix factorization model using the [MovieLens 100K dataset](https://grouplens.org/datasets/movielens/100k/) with TFRS. We can use this model to recommend movies for a given user.

### Import TFRS

First, install and import TFRS:

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2022-05-29 19:48:58.383531: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-29 19:48:58.383549: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Read the data

In [3]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [6]:
ratings = tfds.load('movielens/100k-ratings', split="train")

list(ratings.take(1).as_numpy_iterator())

2022-05-29 19:49:34.315707: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


[{'bucketized_user_age': 45.0,
  'movie_genres': array([7]),
  'movie_id': b'357',
  'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
  'raw_user_age': 46.0,
  'timestamp': 879024327,
  'user_gender': True,
  'user_id': b'138',
  'user_occupation_label': 4,
  'user_occupation_text': b'doctor',
  'user_rating': 4.0,
  'user_zip_code': b'53211'}]

Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [6]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

### Define a model

We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

In [7]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [8]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:



In [10]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 4s 152ms/step - factorized_top_k/top_1_categorical_accuracy: 2.9000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0105 - factorized_top_k/top_50_categorical_accuracy: 0.0730 - factorized_top_k/top_100_categorical_accuracy: 0.1560 - loss: 33232.7133 - regularization_loss: 0.0000e+00 - total_loss: 33232.7133
Epoch 2/3
25/25 [==============================] - 4s 152ms/step - factorized_top_k/top_1_categorical_accuracy: 5.3000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0075 - factorized_top_k/top_10_categorical_accuracy: 0.0204 - factorized_top_k/top_50_categorical_accuracy: 0.1350 - factorized_top_k/top_100_categorical_accuracy: 0.2556 - loss: 30591.7447 - regularization_loss: 0.0000e+00 - total_loss: 30591.7447
Epoch 3/3
25/25 [==============================] - 4s 153ms/step - factorized_top_k/top_1_categorical_accuracy: 3.7000e-04 - factorized_top_k/top_5_categorica

In [11]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']


In [29]:
scores, titles = index(np.array(["1111111111111111111"]))
scores

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.56814724, 0.50953436, 0.44852608, 0.43461978, 0.4092513 ,
        0.39631125, 0.38252503, 0.37978685, 0.37868538, 0.3747409 ]],
      dtype=float32)>

In [30]:
titles

<tf.Tensor: shape=(1, 10), dtype=string, numpy=
array([[b'Catwalk (1995)', b'Scout, The (1994)', b'Bonheur, Le (1965)',
        b'Dracula: Dead and Loving It (1995)', b'Theodore Rex (1995)',
        b'Country Life (1994)', b'High School High (1996)',
        b'Heavy (1995)', b'Feeling Minnesota (1996)',
        b'Secret Adventures of Tom Thumb, The (1993)']], dtype=object)>